# Classfication

This notebook evaluates methods for classification using the [academia.stackexchange.com](https://academia.stackexchange.com/) data dump.

## Table of Contents
* [Data import](#data_import)
* [Classfication methods](#methods)

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from joblib import dump, load
from academia_tag_recommender.definitions import MODELS_PATH

<a id='data_import'/>

## Data import

<a id='methods'/>

## Classification methods

Probabilistic algorithms (high dimensionality, data sparsity)
- Naive Bayes (NB), [Explanation](https://scikit-learn.org/stable/modules/naive_bayes.html)
- [Logistic Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression), [Explanation](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression)
- Linear Classifier (LLSF: Linear Least Squares Fit), [Explanation](https://scikit-learn.org/stable/modules/linear_model.html#ordinary-least-squares)


Geometric algorithms
- [k-Nearest Neighbor (kNN)](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier), [Explanation](https://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors-classification)
- [Support Vector Machine (SVM)](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC), [Explanation](https://scikit-learn.org/stable/modules/svm.html)


- [Neural Network (NN)](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier), [Explanation](https://scikit-learn.org/stable/modules/neural_networks_supervised.html)
